# Processetti
<br>

## Librerie

>   Importazione

In [1]:
import json
import pprint
from collections import Counter
from operator import itemgetter
import csv
from dotmap import DotMap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

-----------------

## Funzioni locali

In [2]:
# pulizzia delle stringhe

def clean_up_str(string):
    a = ''
    if string:
        a = string.replace('\xa0', '').replace('  ', ' ').strip()
    return a  

In [3]:
# Funzione di elminazione delle dupplicazioni in leggenda
##  https://stackoverflow.com/questions/19385639/duplicate-items-in-legend-in-matplotlib

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

-------------

## Preparazione dei dataset


### Overwiev Processetti



#### Appertura e ispezione dei file 

In questa sezione i dati vengono ispezionati una prima volta. Vengono importati gli _output_ scaricati da Geovistory 

I file contengono il risultato di un'interrogazione fatta in Geovistory, una tabella del dominio Analysis, con lo stesso nome, fatta nella data indicata nel nome del file. Se si vuole utilizzare un nuovo file, contenente nuovi dati, va scaricarlo da Geovistory, caricato nella cartella *geovistory_query_exports* e indicare l'indirizzo corretto nella cella seguente

In [9]:
file_pro = 'geovistory_query_exports/nb_overwiev_processetti_20220510.json'

In [10]:
# Aprire il file e attribirlo alla variabile *data* in forma di dizionario Python

with open(file_pro, encoding='utf-8') as json_file:
    data_pro = json.load(json_file)

In [11]:
# Creare un oggetto DotMap con lo scopo di facilitare il lavoro sul file JSON
dm_pro = DotMap(data_pro)

In [12]:
# Creare una variabile contenente le righe
dm_pro_r = dm_pro.rows

> Riprendi descrizione
### Trasformazione e messa in piano dei dati

È necessario disporre di dati trasformati e misurati (sopprimendo in particolare i valori multipli per una singola variabile) per poter effettuare le analisi.

La seguente procedura è stata applicata per creare una nuova tabella:
- se esistevano più anni (per esempio l'anno di nascita), è stata scelta la media
- se esistevano più occupazioni, tutte sono state mantenute e separate con un segno "#".
- se esistevano più tipi di processetto, tutti sono stati mantenuti e separati con un segno "#".


Qui sotto trovate un estratto di questa nuova tabella. Questa nuova tabella viene esportata automaticamente come documento csv. Puoi accedere e scaricare questo documento nel tuo jupyter-binder. Puoi aprirlo in Excel (via csv-import) e analizzarlo anche lì. Questo potrebbe aiutare a capire la struttura della tabella appena creata.

Tradotto con www.DeepL.com/Translator (versione gratuita)
> Riprendi descrizione

In [13]:
data_pro['columns']

{'col_0': 'Volume',
 'col_1': 'Unità archivistiche',
 'col_2': 'Processetto di',
 'col_3': 'Motivation Type',
 'col_4': 'date',
 'col_5': 'Union'}

In [16]:
### Processetti list = psl

psl=[['volume', 'section', 'number_archival_units', 'id_enquiry', 'enquiry', 'motivation_type',
 'start_date','end_date', 'anno', 'id_union', 'number_union']]


for r in dm_pro_r:
    element = []
    # volume
    element += ['-'.join([v.entity_label for v in r.col_0.entities])]   

    # sezione (unità archivistica)
    element += [clean_up_str(r.col_1.entities[0].entity_label)]

    # numero di section per enquiry 
    element += [len(r.col_1.entities)]

    # enquiry 
    element += [r.col_2.entity.pk_entity, clean_up_str(r.col_2.entity.entity_label)]
   
    # mtivation type
    element += [clean_up_str('; '.join([v.entity_label for v in r.col_3.entities]))]

    # arco e anno processetto
    valor = []
    if len(r.col_4['values']) == 0:
        element += [np.nan, np.nan, np.nan]
    else:
        for v in  r.col_4['values']:
            valor.append(int(v.value.timePrimitive.label[:4]))
        element += [r.col_4['values'][0]['value'].timePrimitive.label[:10], r.col_4[
            'values'][len(r.col_4['values'])-1]['value'].timePrimitive.label[:10], round(np.mean(valor))]

    # union
    if len(r.col_5.entities)==1:
        element += [r.col_5.entities[0].pk_entity, len(r.col_5.entities)]
        
    elif len(r.col_5.entities)>1:
        element += ['-'.join([str(v.pk_entity) for v in r.col_5.entities]), len(r.col_5.entities)]  
        print(element)
    else:
        element += [np.nan, 0]
        
    psl.append(element)
    

proc_df = pd.DataFrame(psl[1:], columns=psl[0])  
proc_df.head()
    

        
   


['EM 04', 'Processetto di Angela vedova di Francesco Giovanni Ochiati (da Venezia)', 1, 1270509, 'Angela vedova di Francesco Giovanni Ochiati (da Venezia)', 'Vedovanza', '1597-08-05', '1597-08-05', 1597, '1270524-1270537', 2]
['EM 16', 'Processetto di Sebastiano di Gottardo Murari (da Verona) e Barbara di Antonio Furlani (veronese)', 1, 1986329, 'Sebastiano di Gottardo Murari (da Verona)', 'Vedovanza', '1620-01-12', '1620-01-12', 1620, '1986337-1986343', 2]
['EM 16', 'Processetto di Caterina vedova di Manolo Manuelli (cretese)', 1, 2029147, 'Caterina vedova di Manolo Manuelli (cretese)', 'Vedovanza', '1620-02-15', '1620-02-15', 1620, '2029154-2107629', 2]
['EM 17', 'Processetto di Marietta vedova di Ruggero Perei (moliner)', 1, 2215939, 'Marietta vedova di Ruggero Perei (moliner)', 'Vedovanza', '1620-10-02', '1620-10-02', 1620, '2215946-2215952', 2]


,volume,section,number_archival_units,id_enquiry,enquiry,motivation_type,start_date,end_date,anno,id_union,number_union
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
3413,EM 05,"Processetto di Gaspare Forzanini (da Bassano, ...",1,998606,"Gaspare Forzanini (da Bassano, calegher)",Stato libero di “foresto”,1598-12-18,1598-12-18,1598.0,NaN,0
3414,EM 04,Processetto di Cristina vedova di Nicola De Ga...,1,998615,Cristina vedova di Nicola De Gaspari (da Venezia),Vedovanza,1597-05-20,1597-05-20,1597.0,998630,1
3415,EM 04,Processetto di Giovanni Maria di Antonio (da L...,1,998913,"Giovanni Maria di Antonio (da Lovere, tesser d...",Vedovanza,1597-05-21,1597-05-21,1597.0,998922,1
3416,EM 05,Processetto di Andrea di Angelo Pasetto (da Ma...,1,999218,"Andrea di Angelo Pasetto (da Magno, erbariol)",Stato libero di “foresto”,1598-12-21,1598-12-21,1598.0,NaN,0


> Da sviluppare sulla base delle esigenze di ricerca

### Richiedenti

> __Riprendi da qui__

In [ ]:
file_ric = 'geovistory_query_exports/nb_richiedenti_20220510.json'

with open(file_ric, encoding='utf8') as json_file:
    data_ric = json.load(json_file)

dm_ric = DotMap(data_ric)

dm_ric_r = dm_ric.ro ## Riprendi da qui 

### Testimoni

### Defunti

> Richiede una diversa elaboraizone 

---------------------